In [34]:
import pandas as pd
from pandasql import sqldf

# sqldf("""
# SELECT * 
# FROM df
# LIMIT 5
# """)

In [35]:
df = pd.read_pickle(r'/Users/maya/Documents/Information Systems/שנה ג׳/למידת מכונה/פרוייקט קורס/XY_train.pkl')

# Feature extraction ideas

In [5]:
df.head()

,textID,text,sentiment,message_date,account_creation_date,previous_messages_dates,date_of_new_follower,date_of_new_follow,email,gender,email_verified,blue_tick,embedded_content,platform
0,b8f4e560fa,but yeah i like purple maybe thats why!! ;) ...,positive,2023-10-2 7:41:52,2015-4-13 0:1:16,"[2019-2-15 16:12:42, 2020-6-2 16:49:53, 2021-4...","[2016-1-13 7:12:26, 2016-5-17 13:40:38, 2016-1...","[2016-7-15 0:3:50, 2016-4-10 22:31:8, 2016-10-...",ClareRosindill3265@messenger.gov,F,NaN,False,mp4,facebook
1,f81a1511b2,_Henrie haha i WISH i coudl meet you.. you sho...,positive,2022-11-17 2:1:57,2013-7-1 18:58:16,"[2021-7-5 21:53:48, 2020-12-12 9:7:50, 2020-9-...","[2016-10-4 7:49:46, 2016-6-10 7:47:39, 2016-4-...","[2016-7-5 20:23:57, 2016-5-3 15:9:51, 2018-11-...",ErenaAntonchik231@messenger.gov,M,True,True,False,tiktok
2,3e9e3f0d69,"nah, you`re just altered forever Enjoy.",positive,2022-6-1 6:53:18,2013-8-8 2:21:12,"[2021-7-11 9:59:36, 2020-2-18 6:38:45, 2021-5-...","[2018-7-13 17:0:3, 2016-2-13 17:8:12, 2016-2-1...","[2016-7-9 22:15:10, 2018-4-13 20:37:55, 2016-1...",GodardCowlas6687@api.gov,F,True,True,False,tiktok
3,a068b95bd9,Can you ask Ryan why he stopped following me ...,negative,2022-1-4 11:38:57,2015-10-16 14:51:19,"[2021-8-16 4:55:32, 2021-6-1 10:13:37, 2021-12...",[2018-7-1 5:9:39],"[2018-10-18 7:23:20, 2018-4-10 12:17:6, 2016-6...",RalfStolberger6467@python.gov,M,False,False,mp4,tiktok
4,e6da2d1835,New issue of in the office...desperately want...,negative,2022-5-11 21:56:27,2013-12-15 19:8:40,"[2021-9-11 10:55:6, 2021-5-6 12:1:54, 2021-1-1...","[2018-11-2 11:16:12, 2018-10-18 16:8:34]","[2018-4-18 20:56:34, 2018-7-1 14:56:33, 2016-4...",MadelLeaton248@bbc.gov.il,M,False,False,None,None


In [115]:
# Is edu suffix or not
df['is_edu'] = df.email.str.find('edu') != -1

# Is a man or woman
df['is_man'] = df.gender == 'M'

# Is the email verified 
df['is_email_verified'] = df['email_verified'] == True

# Does the account have a blue tick
df['is_blue_tick'] = df['blue_tick'] == True

# Is embeded content (mp4 or link)
embeded_content_dummies = pd.get_dummies(df.embedded_content)[['link','mp4']].rename(columns={'link':'is_link','mp4':'is_mp4'})
df = pd.concat([df,embeded_content_dummies],axis=1)

# Platform (Instagram or Telegram)
platform_dummies = pd.get_dummies(df.platform)[['instagram','telegram']].rename(columns={'instagram':'is_instagram','telegram':'is_telegram'})
df = pd.concat([df,platform_dummies],axis=1)

################## Chane Y variable to Boolean ##################
df['is_positive_sentiment'] = df_for_model_for_mode['sentiment'] == 'positive'

In [36]:

# Convert 'message_date' and 'account_creation_date' columns to datetime objects
df['message_date'] = pd.to_datetime(df['message_date'])
df['account_creation_date'] = pd.to_datetime(df['account_creation_date'])

# Calculate tenure by subtracting account_creation_date from message_date
df['tenure'] = df['message_date'] - df['account_creation_date']

# Add the value of the sentiment to the DataFrame
# Assuming the sentiment is already present in the DataFrame as 'sentiment'
# If not, you can add it from your source
# For example, if 'sentiment' is stored in a separate pickle file
# sentiment_df = pd.read_pickle('path_to_sentiment_file.pkl')
# df['sentiment'] = sentiment_df['sentiment']
df['tenure_years'] = df['tenure_days'] / 365.25

# Display the DataFrame with tenure and sentiment
print(df[['message_date', 'account_creation_date', 'tenure', 'sentiment']])

KeyError: 'tenure_days'

In [ ]:
import pandas as pd

# Assuming you already have pandas imported and your DataFrame is loaded
# df = pd.read_pickle(r'/Users/maya/Documents/Information Systems/שנה ג׳/למידת מכונה/פרוייקט קורס/XY_train.pkl')

# List of binary columns
binary_columns = ['email', 'gender', 'email_verified', 'blue_tick', 'embedded_content', 'platform']

# Create dummy variables for binary columns
dummy_df = pd.get_dummies(df[binary_columns], columns=binary_columns)

# Concatenate dummy variables with sentiment column
dummy_df['sentiment'] = df['sentiment']

# Perform cross-tabulation
result_df = pd.crosstab(index=[dummy_df[col] for col in dummy_df.columns], columns=dummy_df['sentiment'])

# Add 'total' row and column
result_df['total'] = result_df.sum(axis=1)
result_df.loc['total'] = result_df.sum()

# Rename the index levels and columns
result_df.index.names = [col.replace('_', '') for col in dummy_df.columns]
result_df.columns = ['false', 'true', 'total']

# Reset index to convert index levels to columns
result_df.reset_index(inplace=True)

# Display the result DataFrame
print(result_df)


In [116]:
model_features_list = ['textID','is_edu', 'is_man', 
                       'is_email_verified', 'is_blue_tick',
                       'is_link','is_mp4', 'is_positive_sentiment',
                       'is_instagram','is_telegram']

df_for_model = df[model_features_list].set_index('textID')


In [117]:
df_for_model

,is_edu,is_man,is_email_verified,is_blue_tick,is_link,is_link,is_mp4,is_mp4,is_positive_sentiment,is_instagram,is_instagram,is_telegram,is_telegram
textID,,,,,,,,,,,,,
b8f4e560fa,False,False,False,False,False,False,True,True,True,False,False,False,False
f81a1511b2,False,True,True,True,False,False,False,False,True,False,False,False,False
3e9e3f0d69,False,False,True,True,False,False,False,False,True,False,False,False,False
a068b95bd9,False,True,False,False,False,False,True,True,False,False,False,False,False
e6da2d1835,False,True,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35ee036565,False,False,False,False,False,False,False,False,True,False,False,False,False
10c23a0f46,False,False,True,True,False,False,False,False,True,False,False,False,False
a2dd416423,False,False,True,False,True,True,False,False,False,False,False,True,True


In [121]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assume df_for_model is your pandas DataFrame with X variables and the target variable 'is_positive_sentiment'

# Step 1: Split the data into features (X) and target variable (y)
X = df_for_model.drop('is_positive_sentiment', axis=1)
y = df_for_model['is_positive_sentiment']

# Step 2: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Import the logistic regression model
# Already imported in the previous step

# Step 4: Train the logistic regression model
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = logistic_regression_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.7307535641547862


In [150]:
import statsmodels.api as sm

# Given Pandas DataFrame
# Assume df is your pandas DataFrame with X variables and the target variable 'is_positive_sentiment'

# Step 1: Split the data into features (X) and target variable (y)
X = df[['is_edu', 'is_man', 'is_email_verified', 'is_blue_tick', 'is_link', 'is_mp4', 'is_instagram', 'is_telegram']]
y = df['is_positive_sentiment']

# Step 2: Add constant to the features (X) - this is required for statsmodels
X = sm.add_constant(X)

# Step 3: Create and fit the logistic regression model
logit_model = sm.Logit(y.values, X.values)
logit_result = logit_model.fit()

# Step 4: Get summary of the model
print(logit_result.summary())

# Step 5: Check p-values and eliminate non-significant variables
significant_vars = logit_result.pvalues[logit_result.pvalues < 0.05].index
X_significant = X[significant_vars]

# Step 6: Refit the logistic regression model with significant variables
logit_model_significant = sm.Logit(y, X_significant)
logit_result_significant = logit_model_significant.fit()

# Step 7: Get summary of the new model with significant variables
print(logit_result_significant.summary())

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

(12272, 13)

In [104]:
# pd.DataFrame.from_dict(df['text'][:10].to_dict())

pd.Series({0: ' but yeah i like purple maybe thats why!! ;)  :p :d',
 1: '_Henrie haha i WISH i coudl meet you.. you should stop by seattle some time  home of the STARBUKS ;) I LOVE YOU DAVID!!',
 2: ' nah, you`re just altered forever   Enjoy.',
 3: ' Can you ask Ryan why he stopped following me on Twitter',
 4: 'New issue of  in the office...desperately want to flick through but have so much to do. It`ll have to wait',
 5: ' very very cute  and fun to watch.',
 6: '  Enjoy...wish I was there! Nice day in Springfield today..however I have my end of month expense report to do',
 7: ' Sadly, all I have is the Stanley Steemer 800 number.',
 8: '  you are disappointing me......',
 9: ' yes! fb, ah i miss you'})


0     but yeah i like purple maybe thats why!! ;)  ...
1    _Henrie haha i WISH i coudl meet you.. you sho...
2            nah, you`re just altered forever   Enjoy.
3     Can you ask Ryan why he stopped following me ...
4    New issue of  in the office...desperately want...
5                    very very cute  and fun to watch.
6      Enjoy...wish I was there! Nice day in Spring...
7     Sadly, all I have is the Stanley Steemer 800 ...
8                       you are disappointing me......
9                               yes! fb, ah i miss you
dtype: object